In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
import keras.backend as K
import tensorflow as tf 
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config = config)
K.set_session(sess)

In [5]:
from keras.models import Model
from keras.layers import Input,LSTM,Dense
import numpy as np 

In [6]:
batch_size = 64
epochs = 10 
latent_dim = 256
num_samples = 10000
data_path = r'data/fra.txt'

In [7]:
#vectorize the data
input_texts = []
target_texts =[]
input_characters = set()
target_characters = set()
with open(data_path,'r',encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[:min(num_samples,len(lines)-1)]:
    input_text , target_text = line.split('\t')
    #'\t' 作为start sequence '\n' 作为 end sequence
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    
    #建立词表vocab
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

#对vocab排了个序 没必要
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
#vocab size 
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)

#文本最大长度
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])
print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 10000
Number of unique input tokens: 70
Number of unique output tokens: 93
Max sequence length for inputs: 16
Max sequence length for outputs: 59


In [ ]:
#字典映射 char - id 
input_token_index = dict([char ,i] for i,char in enumerate(input_characters))
target_token_index = dict([char,i] for i,char in enumerate(target_characters))

encoder_input_data = np.zeros((len(input_texts),max_encoder_seq_length,num_encoder_tokens),dtype = 'float32')
decoder_input_data = np.zeros((len(input_texts),max_decoder_seq_length,num_decoder_tokens),dtype = 'float32')
decoder_target_data = np.zeros((len(input_texts),max_decoder_seq_length,num_decoder_tokens),dytpe = 'float32')


